In [50]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.metrics import roc_curve, auc,recall_score,precision_score

In [8]:
df = pd.read_csv(r"oof_set.csv",index_col=0)
df

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke,gender_v,ever_married_v,work_type_v,Residence_type_v,smoking_status_v,predicted_XGB_stroke,predicted_LGBM_stroke,predicted_CAT_stroke
0,0,Male,28.0,0,0,Yes,Private,Urban,79.53,31.1,never smoked,0,1,1,4,1,1,0.021125,0.015956,0.009944
1,1,Male,33.0,0,0,Yes,Private,Rural,78.44,23.9,formerly smoked,0,1,1,4,0,0,0.022357,0.014877,0.009142
2,2,Female,42.0,0,0,Yes,Private,Rural,103.00,40.3,Unknown,0,0,1,4,0,4,0.025423,0.020765,0.022139
3,3,Male,56.0,0,0,Yes,Private,Urban,64.87,28.8,never smoked,0,1,1,4,1,1,0.048896,0.039536,0.031538
4,4,Female,24.0,0,0,No,Private,Rural,73.36,28.8,never smoked,0,0,0,4,0,1,0.016632,0.009917,0.008360
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15299,15299,Female,22.0,0,0,No,Govt_job,Urban,72.63,19.5,never smoked,0,0,0,2,1,1,0.016956,0.006834,0.004107
15300,15300,Female,46.0,1,0,Yes,Private,Urban,101.19,32.1,never smoked,0,0,1,4,1,1,0.046671,0.023077,0.020201
15301,15301,Female,75.0,0,0,Yes,Self-employed,Urban,87.69,26.2,never smoked,0,0,1,1,1,1,0.124246,0.148357,0.161940
15302,15302,Male,46.0,0,0,Yes,Private,Rural,101.13,22.5,Unknown,0,1,1,4,0,4,0.024573,0.017648,0.009523


In [12]:
def roc_auc_score(y_valid, preds):
    fpr, tpr, _ = roc_curve(y_valid, preds)
    roc_auc = auc(fpr, tpr)
    return roc_auc

In [52]:
combos = []

for x in range(100):
    for y in range(100):
        if x + y < 100:
            z = 100 - x - y
            combos.append([x, y, z])
combos = np.array(combos) * ([[0.01]*3]*len(combos))
len(combos)

5050

In [53]:
coefficients = combos
best_score = 0
best_coefficients_index = 0

for index, (a, b, c) in tqdm(enumerate(coefficients)):
    score = roc_auc_score(df["stroke"], (a * df['predicted_XGB_stroke']) + (b * df['predicted_LGBM_stroke']) + (c * df['predicted_CAT_stroke']))
    if score > best_score:
        best_score = score
        best_coefficients_index = index

5050it [00:09, 549.40it/s]


In [54]:
best_coefficients_index

5044

In [55]:
best_score


0.9101666568888643

In [56]:
combos[best_coefficients_index]

array([0.97, 0.  , 0.03])

In [42]:
combos

array([[0.   , 0.   , 1.   ],
       [0.   , 0.001, 0.999],
       [0.   , 0.002, 0.998],
       ...,
       [0.998, 0.   , 0.002],
       [0.998, 0.001, 0.001],
       [0.999, 0.   , 0.001]])